In [1]:
# First import library
import pyrealsense2 as rs
# Import Numpy for easy array manipulation
import numpy as np
# Import OpenCV for easy image rendering
import cv2
import open3d as o3d
from datetime import datetime

def get_intrinsic_matrix(frame):
    intrinsics = frame.profile.as_video_stream_profile().intrinsics
    out = o3d.camera.PinholeCameraIntrinsic(640, 480, intrinsics.fx,
                                            intrinsics.fy, intrinsics.ppx,
                                            intrinsics.ppy)
    return out


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Create pipeline
pipeline = rs.pipeline()

# Create a config object
config = rs.config()

# Tell config that we will use a recorded device from file to be used by the pipeline through playback.
rs.config.enable_device_from_file(config, "20230404_112125.bag")
#rs.config.enable_device_from_file(config, "20230403_112453.bag")

In [ ]:
config.enable_stream(rs.stream.depth, rs.format.z16, 30)
pipeline.start(config)

#cv2.namedWindow("Depth Stream", cv2.WINDOW_AUTOSIZE)
vis = o3d.visualization.Visualizer()
vis.create_window()

view_control = vis.get_view_control()
view_control.set_lookat([2.6172, 2.0475, 1.532])
view_control.set_front([0.4257, -0.2125, -0.8795])
view_control.set_zoom(0.3412)
view_control.set_up([-0.0694, -0.9768, 0.2024])

colourizer = rs.colorizer()

count = 0

while True:
    frames = pipeline.wait_for_frames()
    print(f"Got frame {frames}")
    depth_frame = frames.get_depth_frame()
    colour_frame = frames.get_color_frame()
    
    intrinsic = o3d.camera.PinholeCameraIntrinsic(
                    get_intrinsic_matrix(depth_frame))
    print(f"Colour frame {colour_frame}")
    if colour_frame:
        print("OK frame")
    else:
        print("Bad frame")

    #if not colour_frame:
    #    colour_frame = o3d.geometry.Image()
    if colour_frame:
        print("Yeah - valid colour frame!")
        intrinsic = o3d.camera.PinholeCameraIntrinsic(
                        get_intrinsic_matrix(colour_frame))
        #depth_image = o3d.geometry.Image(
        #                  np.array(aligned_depth_frame.get_data()))
        color_temp = np.array(color_frame.get_data())
        color_image = o3d.geometry.Image(color_temp)

        rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
                color_image,
                depth_image,
                depth_scale=1.0 / depth_scale,
                depth_trunc=clipping_distance_in_meters,
                convert_rgb_to_intensity=False)
        temp = o3d.geometry.PointCloud.create_from_rgbd_image(
                rgbd_image, intrinsic)
        #o3d.visualization.draw_geometries([temp],zoom=0.5)
        o3d.visualization.update_geometry(temp)

    depth_colour_frame = colourizer.colorize(depth_frame)

    align_to = rs.stream.color
    align = rs.align(align_to)

    # Align the depth frame to color frame
    aligned_frames = align.process(frames)

    # Get aligned frames
    aligned_depth_frame = aligned_frames.get_depth_frame()
    depth_image = o3d.geometry.Image(np.array(aligned_depth_frame.get_data()))

    pcd = o3d.geometry.PointCloud.create_from_depth_image(depth_image, intrinsic,
                                                         np.identity(4), depth_scale=1000.0, depth_trunc=3000.0)

    mean,cov = pcd.compute_mean_and_covariance()
    
    print(f"Point cloud {count}: {mean} {cov} {pcd}")
    # This is supposed to work - but the pcd is the wrong type?
    #o3d.visualization.draw_geometries([pcd],zoom=0.5)
    if count == -1:
        # This blocks until the window is closed
        o3d.visualization.draw_geometries([pcd],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=mean,
                                  up=[-0.0694, -0.9768, 0.2024])
    else:
        # Update geometry doesn't seem to honour the view control changes
        #vis.update_geometry(pcd)
        vis.add_geometry(pcd)
        
        # The add_geometry call changes the view control, so we need to read it after we make t
        view_control = vis.get_view_control()
        view_control.set_lookat(mean)
        view_control.set_front([0.4257, -0.2125, -0.8795])
        view_control.set_zoom(0.3412)
        view_control.set_up([-0.0694, -0.9768, 0.2024])

        t0 = datetime.now()
        t = datetime.now()-t0
        while t.total_seconds() < 3.0:
            vis.poll_events()
            vis.update_renderer()
            t = datetime.now()-t0
    count = count + 1
    #o3d.visualization.draw_geometries([temp],zoom=0.5)

    #depth_image = o3d.geometry.Image(
    #            np.array(aligned_depth_frame.get_data()))
    #depth_colour_frame = colourizer.colorize(depth_frame)

    #cv2.show("Depth ",depth_colour_frame)
    print(f"Got frame {depth_frame}")
    #key = cv2.waitKey(1)
    key = 0
    if key == 27:
            cv2.destroyAllWindows()
            break

Got frame <pyrealsense2.frameset Z16 #773 @1680621685633.607178>
Colour frame <pyrealsense2.frame NULL>
Bad frame
Point cloud 0: [ 0.54041125 -0.43062582  2.09234014] [[ 1.56508717  0.02744785  0.75938059]
 [ 0.02744785  0.62437441 -1.18658434]
 [ 0.75938059 -1.18658434  4.67050932]] PointCloud with 336566 points.
Got frame <pyrealsense2.frame Z16 #773 @1680621685633.607178>
Got frame <pyrealsense2.frameset Z16 #863 @1680621688634.559326>
Colour frame <pyrealsense2.frame NULL>
Bad frame
Point cloud 1: [ 0.02055414 -0.42719854  1.78615599] [[ 4.6470038  -0.71496639  1.20117041]
 [-0.71496639  0.70474708 -1.76760971]
 [ 1.20117041 -1.76760971  6.56923219]] PointCloud with 313498 points.
Got frame <pyrealsense2.frame Z16 #863 @1680621688634.559326>
Got frame <pyrealsense2.frameset Z16 #954 @1680621691668.909424>
Colour frame <pyrealsense2.frame NULL>
Bad frame
Point cloud 2: [-0.71113879 -0.35989124  1.82952881] [[ 3.18460162  0.47059088 -3.50429764]
 [ 0.47059088  0.78132288 -1.61587845]

Got frame <pyrealsense2.frame Z16 #1017 @1680621693769.631104>
Got frame <pyrealsense2.frameset Z16 #830 @1680621687534.197266>
Colour frame <pyrealsense2.frame NULL>
Bad frame
Point cloud 22: [ 0.33340714 -0.45739991  1.68306791] [[ 1.38862816 -0.35172667  1.35273035]
 [-0.35172667  0.56500564 -1.3853223 ]
 [ 1.35273035 -1.3853223   4.53135636]] PointCloud with 312538 points.
Got frame <pyrealsense2.frame Z16 #830 @1680621687534.197266>
Got frame <pyrealsense2.frameset Z16 #924 @1680621690668.590088>
Colour frame <pyrealsense2.frame NULL>
Bad frame
Point cloud 23: [-0.42601231 -0.30309654  1.30769682] [[ 1.77473092  0.71718464 -2.48488429]
 [ 0.71718464  0.53142739 -1.40641941]
 [-2.48488429 -1.40641941  4.63167258]] PointCloud with 336541 points.
Got frame <pyrealsense2.frame Z16 #924 @1680621690668.590088>
Got frame <pyrealsense2.frameset Z16 #1014 @1680621693669.597900>
Colour frame <pyrealsense2.frame NULL>
Bad frame
Point cloud 24: [-0.76234971 -0.4999217   2.53841662] [[ 3.77117